In [35]:
class str2(str):
    def __repr__(self):
        return ''.join(('"', super().__repr__()[1:-1], '"'))

In [50]:
path = 'google/muril-base-cased'
## Loading the model
from transformers import AutoModel, AutoTokenizer
import torch
import json
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path,output_hidden_states=True)
## Embeddings 
# text = ["कोई अच्छी सी फिल्म लगायो", "कोई"]
text = ['कोई', 'अच्छी', 'सी']
# text = [str2(token) for token in text]
print(text)
print(tokenizer.convert_ids_to_tokens(tokenizer.encode(text)))
input_encoded = tokenizer.encode_plus(text, return_tensors="pt")
with torch.no_grad():
         states = model(**input_encoded).hidden_states
output = torch.stack([states[i] for i in range(len(states))])
output = output.squeeze()
print("Output shape is {}".format(output.shape))

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['कोई', 'अच्छी', 'सी']


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [3]:
import numpy as np
class InputFeature(object):
    def __init__(self,
                 question,
                 table_id,
                 tokens,
                 word_to_char_start,
                 word_to_subword,
                 subword_to_word,
                 input_ids,
                 input_mask,
                 segment_ids):
        self.question = question
        self.table_id = table_id
        self.tokens = tokens
        self.word_to_char_start = word_to_char_start
        self.word_to_subword = word_to_subword
        self.subword_to_word = subword_to_word
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids

        self.columns = None
        self.agg = None
        self.select = None
        self.where_num = None
        self.where = None
        self.op = None
        self.value_start = None
        self.value_end = None

    def output_SQ(self, agg = None, sel = None, conditions = None, return_str=True):
        agg_ops = ['NA', 'MAX', 'MIN', 'COUNT', 'SUM', 'AVG']
        cond_ops = ['=', '>', '<', 'OP']

        if agg is None and sel is None and conditions is None:
            sel = np.argmax(self.select)
            agg = self.agg[sel]
            conditions = []
            for i in range(len(self.where)):
                if self.where[i] == 0:
                    continue
                conditions.append((i, self.op[i], self.value_start[i], self.value_end[i]))

        agg_text = agg_ops[agg]
        select_text = self.columns[sel]
        cond_texts = []
        for wc, op, vs, ve in conditions:
            column_text = self.columns[wc]
            op_text = cond_ops[op]
            word_start, word_end = self.subword_to_word[wc][vs], self.subword_to_word[wc][ve]
            char_start = self.word_to_char_start[word_start]
            char_end = len(self.question) if word_end + 1 >= len(self.word_to_char_start) else self.word_to_char_start[word_end + 1]
            value_span_text = self.question[char_start:char_end]
            cond_texts.append(column_text + op_text + value_span_text.rstrip())

        if return_str:
            sq = agg_text + ", " + select_text + ", " + " AND ".join(cond_texts)
        else:
            sq = (agg_text, select_text, set(cond_texts))

        return sq

In [52]:
path = 'google/muril-base-cased'
## Loading the model
from transformers import AutoModel, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModel.from_pretrained(path,output_hidden_states=True)

question = "कोई अच्छी सी"
table_id = "1-10015132-11"
word_to_char_start = [0, 4, 9]
example_tokens = ["कोई", "अच्छी", "सी"]
column_meta = [["Player", "string", None], ["No.", "string", None], ["Nationality", "string", None], ["Position", "string", None], ["Years in Toronto", "string", None], ["School/Club Team", "string", "Butler CC (KS)"]]
max_total_length = 96

input_feature = InputFeature(
    question,
    table_id,
    [],
    word_to_char_start,
    [],
    [],
    [],
    [],
    []
)

for column, col_type, _ in column_meta:
    # get query tokens
    print(column)
    print(col_type)
    tokens = []
    word_to_subword = []
    subword_to_word = []
    for i, query_token in enumerate(example_tokens):
        sub_tokens = tokenizer.tokenize(query_token)
        cur_pos = len(tokens)
        if len(sub_tokens) > 0:
            word_to_subword += [(cur_pos, cur_pos + len(sub_tokens))]
            tokens.extend(sub_tokens)
            subword_to_word.extend([i] * len(sub_tokens))
    print(tokens)
    # tokens = [str2(token) for token in tokens]
    # tokenize_result = tokenizer.encode_plus(
    #     question,
    #     return_tensors="pt")
    tokenize_result = tokenizer.encode_plus(
        col_type + " " + column,
        question,
        padding="max_length",
        max_length=max_total_length,
        truncation_strategy="longest_first",
        truncation=True,
    )

    input_ids = tokenize_result["input_ids"]
    input_mask = tokenize_result["attention_mask"]

    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    column_token_length = 0
    for i, token_id in enumerate(input_ids):
        if token_id == tokenizer.sep_token_id:
            column_token_length = i + 1
            break
    segment_ids = tokenize_result["token_type_ids"]

    subword_to_word = [0] * column_token_length + subword_to_word
    word_to_subword = [(pos[0]+column_token_length, pos[1]+column_token_length) for pos in word_to_subword]

    assert len(input_ids) == max_total_length
    assert len(input_mask) == max_total_length
    assert len(segment_ids) == max_total_length

    input_feature.tokens.append(tokens)
    input_feature.word_to_subword.append(word_to_subword)
    input_feature.subword_to_word.append(subword_to_word)
    input_feature.input_ids.append(input_ids)
    input_feature.input_mask.append(input_mask)
    input_feature.segment_ids.append(segment_ids)


model_inputs = {k: [] for k in ["input_ids", "input_mask", "segment_ids"]}
model_inputs["input_ids"].extend(input_feature.input_ids)
model_inputs["input_mask"].extend(input_feature.input_mask)
model_inputs["segment_ids"].extend(input_feature.segment_ids)

for k in model_inputs:
    model_inputs[k] = np.array(model_inputs[k], dtype=np.int64)
it = {k: torch.from_numpy(model_inputs[k][:]) for k in ["input_ids", "input_mask", "segment_ids"]}
            
print(it["input_ids"].size())
bert_output, pooled_output, _ = model(
                input_ids=it["input_ids"],
                attention_mask=it["input_mask"],
                token_type_ids=it["segment_ids"],
                return_dict=False)
print("bbb", bert_output.shape)
print("ppp", pooled_output.shape)

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Player
string
['कोई', 'अच्छी', 'सी']
No.
string
['कोई', 'अच्छी', 'सी']
Nationality
string
['कोई', 'अच्छी', 'सी']
Position
string
['कोई', 'अच्छी', 'सी']
Years in Toronto
string
['कोई', 'अच्छी', 'सी']
School/Club Team
string
['कोई', 'अच्छी', 'सी']
torch.Size([6, 96])
bbb torch.Size([6, 96, 768])
ppp torch.Size([6, 768])
